# Assignment 1
## Digital Musicology

@authors: Joris Monnet, Xingyu Pan, Yutaka Osaki, Yiwei Liu

Due Date: 24/04/2024


# Task B

From your analysis in task A, think about how to model these aspects computationally. In this task, choose one or more aspect(s) in your analysis you would like to model.

You should write an algorithm that outputs a MIDI file from an musicXML or the unperformed MIDI. In your report, please clearly describe your model.

In [3]:
from src.utils import get_stream

midi_file_path = './Berceuse_op_57/corrected_midi_score.mid'
my_stream = get_stream(midi_file_path)

### Execute all function

In [4]:
from src.articulations import increase_volume_of_higher_notes_in_track
from src.timings import execute_adjust_durations_for_specific_measure, execute_change_duration_in_measure, \
    accelerate_measure

# Convert MIDI file to a music21 stream
modified_score = execute_adjust_durations_for_specific_measure(my_stream, 1, 14)
modified_score = execute_change_duration_in_measure(modified_score, 15, 18)
modified_score = accelerate_measure(modified_score, 19, 1.2)
modified_score = accelerate_measure(modified_score, 20, 0.9)
modified_score = accelerate_measure(modified_score, 21, 1.2)
modified_score = accelerate_measure(modified_score, 22, 0.8)
modified_score = accelerate_measure(modified_score, 23, 1.0)
modified_score = accelerate_measure(modified_score, 24, 1.0)
modified_score = accelerate_measure(modified_score, 25, 1.2)
modified_score = accelerate_measure(modified_score, 26, 1.0)
modified_score = increase_volume_of_higher_notes_in_track(modified_score, 27, 30)

from src.dynamics import randomize_velocity_in_measures
from src.articulations import increase_volume_of_highest_note_in_triples, apply_pedal_to_measures, \
    increase_volume_of_higher_notes_in_track, accentuate_highest_note_in_measure

# 33-34 40-41 high pitch with higher intensity
my_stream = accentuate_highest_note_in_measure(my_stream,37,1.2)
my_stream = increase_volume_of_highest_note_in_triples(my_stream,40,41)
my_stream = increase_volume_of_higher_notes_in_track(my_stream, 45, 46)

# randomly adjust the velocity
my_stream = randomize_velocity_in_measures(my_stream, 31, 46, 2)

# pedals
my_stream = apply_pedal_to_measures(my_stream, 1, 68)

from src.articulations import apply_trill_to_hand_note

# add trills
my_stream = apply_trill_to_hand_note(my_stream, 'right', measure_number=43, note_index=-2, 
                         semitones=1, trill_speed=1/4, trill_duration=1)
my_stream = apply_trill_to_hand_note(my_stream, 'right', measure_number=43, note_index=-1, 
                         semitones=1, trill_speed=1/8, trill_duration=0.5)
my_stream = apply_trill_to_hand_note(my_stream, 'right', measure_number=44, note_index=0, 
                         semitones=2, trill_speed=1/4, trill_duration=1)
my_stream = apply_trill_to_hand_note(my_stream, 'right', measure_number=44, note_index=12, 
                         semitones=2, trill_speed=1/4, trill_duration=1)

# Change Loudness 
from src.dynamics import change_dynamics_for_whole_piece

my_stream = change_dynamics_for_whole_piece(my_stream)

# Save the modified score
from src.utils import save_midi
save_midi(my_stream, 'final_output.mid')

Measure 1 adjusted.
Measure 2 adjusted.
Measure 3 adjusted.
Measure 4 adjusted.
Measure 5 adjusted.
Measure 6 adjusted.
Measure 7 adjusted.
Measure 8 adjusted.
Measure 9 adjusted.
Measure 10 adjusted.
Measure 11 adjusted.
Measure 12 adjusted.
Measure 13 adjusted.
Measure 14 adjusted.
Measure 15 adjusted.
Measure 16 adjusted.
Measure 17 adjusted.
Measure 18 adjusted.
Measure 27 adjusted.
Measure 28 adjusted.
Measure 29 adjusted.
Measure 30 adjusted.
Measure 40 adjusted.
Measure 41 adjusted.
Measure 45 adjusted.
Measure 46 adjusted.


ValueError: max() arg is an empty sequence

## Task C

Compare your generated MIDI with both the unperformed MIDI and the human performace recording. Discuss their differences and potentially what and how you can further improve. This should be part of your discussion section in the report.

In [ ]:
# TODO visualizations of differences ?